<a href="https://colab.research.google.com/github/aswinsuresh-04/Computational_Linguistics/blob/main/Linquistic_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Extract digit from a string


In [ ]:
#Extract digit from a string

string = input("Enter a string: ")
digits = ""

for char in string:
    if char.isdigit():
        digits += char

print("Extracted digits:", digits)

Enter a string: abcd123efg32hij76767
Extracted digits: 1233276767



Extract phone number from a string


In [ ]:
#Extract phone number from a string

def extract_phone_numbers(text):
    digits = ""
    phone_numbers = []

    for char in text:
        if char.isdigit():
            digits += char
        else:
            if len(digits) == 10:
                phone_numbers.append(digits)
            digits = ""

    if len(digits) == 10:
        phone_numbers.append(digits)

    return phone_numbers

text = input("Enter a string containing 10-digit phone numbers: ")
result = extract_phone_numbers(text)
if result:
    print("Phone numbers found:", result)
else:
    print("No 10-digit phone number found.")

Enter a string containing 10-digit phone numbers: 1234567890 is my number 0987654321 is his number 0987654323 is her number
Phone numbers found: ['1234567890', '0987654321', '0987654323']


Extract phone number from a string

In [ ]:
#Extract gmail from a string

def extract_email(text):
    emails = []
    current = ""
    in_email = False
    i = 0

    while i < len(text):
        char = text[i]

        # Start collecting if we find a letter, number, or allowed email characters
        if char.isalnum() or char in "@.-_":
            current += char
            in_email = True
        else:
            # If we were building an email and hit a non-email character, check if valid
            if in_email:
                if is_valid_email(current):
                    emails.append(current)
                current = ""
                in_email = False
            # Skip spaces or other non-email characters
        i += 1

    # Check if there's a valid email at the end of the string
    if in_email and is_valid_email(current):
        emails.append(current)

    return emails

def is_valid_email(s):
    # Check for exactly one @ and at least one . after it
    if s.count("@") != 1 or "." not in s.split("@")[1]:
        return False
    # Ensure there's at least one character before and after @
    user, domain = s.split("@")
    if not user or not domain:
        return False
    # Basic check for valid characters (alphanumeric, dots, hyphens, underscores)
    for char in user:
        if not (char.isalnum() or char in ".-_"):
            return False
    for char in domain:
        if not (char.isalnum() or char in ".-"):
            return False
    return True

# Get input from user
text = input("Enter a string containing email addresses: ")
result = extract_email(text)
if result:
    print("Email addresses found:", result)
else:
    print("No valid email addresses found.")

In [ ]:
#20/08/2025 - WEDNESDAY#

Implement simple rule based tokenizer for the English language, using regular expressions. Your tokenizer should consider  punctuations and special symbols as separate tokens. contractions like 'isn't' should be regarded as two tokens: is and n't, also identify abbreviations(eg: USA) and internal hyphenation(ice-cream) as single tokens

In [ ]:
def simple_tokenizer(text):
    tokens = []
    current = ""

    for ch in text:
        if ch.isalnum() or ch in ["-", "'"]:   # letters, numbers, hyphen, apostrophe
            current += ch
        elif ch == ".":
            if current and current[-1].isalpha() and len(current) == 1:
                # case like "U.S.A."
                current += "."
            elif current and current[-1] == ".":
                # continuation of abbreviation
                current += "."
            elif current and "." in current:
                # already abbreviation like U.S.A.
                current += "."
            else:
                if current:
                    tokens.append(current)
                    current = ""
                tokens.append(".")
        else:
            if current:
                tokens.append(current)
                current = ""
            if not ch.isspace():
                tokens.append(ch)

    if current:
        tokens.append(current)

    # Fix contractions like isn't -> is + n't
    final_tokens = []
    for token in tokens:
        if "'" in token and not token.startswith("'"):
            if token.endswith("n't"):  # isn't, don't, can't
                final_tokens.append(token[:-3])
                final_tokens.append("n't")
            else:
                idx = token.index("'")
                first, second = token[:idx], token[idx:]
                if first:
                    final_tokens.append(first)
                final_tokens.append(second)
        else:
            final_tokens.append(token)

    return final_tokens


# Let user enter input
text = input("Enter a sentence: ")
print(simple_tokenizer(text))


Enter a sentence: John isn't sure if Dr. Smith lives in the U.S.A., but he's certain that ice-cream, well-known desserts, and cakes are popular there!
['John', 'is', "n't", 'sure', 'if', 'Dr', '.', 'Smith', 'lives', 'in', 'the', 'U.S.A.', ',', 'but', 'he', "'s", 'certain', 'that', 'ice-cream', ',', 'well-known', 'desserts', ',', 'and', 'cakes', 'are', 'popular', 'there', '!']


Write a program or a script that corrects a single non-word spelling error based on the noisy channel model.

In [ ]:
# Noisy Channel Model Spell Correction with Edit Distance

def edit_distance(a, b):
    """Compute Levenshtein edit distance between two words."""
    dp = [[0] * (len(b) + 1) for _ in range(len(a) + 1)]
    for i in range(len(a) + 1):
        for j in range(len(b) + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif a[i-1] == b[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
    return dp[-1][-1]

# Step 1: Input dictionary
dictionary = input("Enter dictionary words (space-separated): ").split()

# Step 2: Input misspelled word
misspelled = input("Enter the misspelled word: ")

# Step 3: Assumptions
P_prior = {w: 1/len(dictionary) for w in dictionary}  # Equal priors

# Step 4: Score calculation
scores = {}
for w in dictionary:
    dist = edit_distance(misspelled, w)
    # Smaller distance = higher probability
    P_s_given_w = 1 / (1 + dist)
    scores[w] = round(P_s_given_w * P_prior[w], 3)

# Step 5: Best correction
best_correction = max(scores, key=scores.get)

# Output
print("\nMisspelled word:", misspelled)
print("Candidates and scores:", scores)
print("Best correction:", best_correction)


Enter dictionary words (space-separated): work word world wreck
Enter the misspelled word: wrok

Misspelled word: wrok
Candidates and scores: {'work': 0.083, 'word': 0.062, 'world': 0.062, 'wreck': 0.083}
Best correction: work


Perform POS Tagging for an Indian lang (hindi since malayalam is too tough) using a pretrained model or dataset.  
Task:
  - Load a small set of sentences in the target lang.
  - Use libraries such as Stanza(?), IndiaNLP, or NLTK to tag parts of speech.
  - Identify common tag types and compare with english tags.
  - Discuss the challenges of POS tagging in morphologically rich lang.


In [6]:
pip install transformers torch


In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# MuRIL model (supports 17 Indian languages including Malayalam)
model_name = "google/muril-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)

text = "ഞാൻ പുസ്തകം വായിക്കുന്നു."  # I am reading a book

result = nlp(text)
for token in result:
    print(token)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


{'entity': 'LABEL_1', 'score': np.float32(0.5048954), 'index': 1, 'word': 'ഞാൻ', 'start': 0, 'end': 3}
{'entity': 'LABEL_1', 'score': np.float32(0.50455445), 'index': 2, 'word': 'പുസ്തകം', 'start': 4, 'end': 11}
{'entity': 'LABEL_1', 'score': np.float32(0.50409645), 'index': 3, 'word': 'വായ', 'start': 12, 'end': 15}
{'entity': 'LABEL_1', 'score': np.float32(0.5049926), 'index': 4, 'word': '##ിക്കുന്നു', 'start': 15, 'end': 24}
{'entity': 'LABEL_1', 'score': np.float32(0.5048647), 'index': 5, 'word': '.', 'start': 24, 'end': 25}


In [9]:
pip install stanza

In [10]:
import stanza

# Download Hindi model (first time only)
stanza.download('hi')

# Load the pipeline
nlp = stanza.Pipeline('hi', processors='tokenize,pos')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: hi (Hindi) ...
INFO:stanza:File exists: /root/stanza_resources/hi/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: hi (Hindi):
| Processor | Package     |
---------------------------
| tokenize  | hdtb        |
| pos       | hdtb_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


In [4]:
sentences = [
    "राम स्कूल जा रहा है।",   # Ram is going to school.
    "सीता आम खा रही है।",   # Sita is eating a mango.
    "यह एक सुंदर बगीचा है।", # This is a beautiful garden.
    "मैं किताब पढ़ता हूँ।"    # I read a book.
]
for sent in sentences:
    doc = nlp(sent)
    print(f"\nSentence: {sent}")
    for word in doc.sentences[0].words:
        print(f"{word.text}\t{word.upos}\t({word.xpos})")



Sentence: राम स्कूल जा रहा है।
राम	PROPN	(NNP)
स्कूल	NOUN	(NN)
जा	VERB	(VM)
रहा	AUX	(VAUX)
है	AUX	(VAUX)
।	PUNCT	(SYM)

Sentence: सीता आम खा रही है।
सीता	PROPN	(NNP)
आम	ADJ	(JJ)
खा	VERB	(VM)
रही	AUX	(VAUX)
है	AUX	(VAUX)
।	PUNCT	(SYM)

Sentence: यह एक सुंदर बगीचा है।
यह	PRON	(PRP)
एक	NUM	(QC)
सुंदर	ADJ	(JJ)
बगीचा	NOUN	(NN)
है	AUX	(VM)
।	PUNCT	(SYM)

Sentence: मैं किताब पढ़ता हूँ।
मैं	PRON	(PRP)
किताब	NOUN	(NN)
पढ़ता	VERB	(VM)
हूँ	AUX	(VAUX)
।	PUNCT	(SYM)
